In [2]:
import pandas as pd
from pandas.core.frame import DataFrame
import numpy as np
import statsmodels.api as sm
import sklearn.model_selection as model_selection

In [3]:
race1 = pd.ExcelFile("C02003_detailedRace_censusTract.xlsx").parse('detailedRace_censusTract')

ethnicity1 = pd.ExcelFile("B03002_ethnicity_censusTract.xlsx").parse('ethnicity_censusTract')

economic1 = pd.ExcelFile("DP03_economic_censusTract.xlsx").parse('economic_censusTract')

education1 = pd.ExcelFile("S1501_education_censusTract.xlsx").parse('education_censusTract')

occupation1 = pd.ExcelFile("S2401_occupation_censusTract.xlsx").parse('occupation_censusTract')

In [4]:
race2 = race1.loc[:,['GEO.id2','HD01_VD01','HD01_VD03']]

ethnicity2 = ethnicity1.loc[:,['GEO.id2','HD01_VD02','HD01_VD12']]

economic2 = economic1.loc[:,['GEO.id2','HC01_VC85']]

education2 = education1.loc[:,['GEO.id2','HC02_EST_VC18']]

occupation2 = occupation1.loc[:,['GEO.id2','HC01_EST_VC08','HC01_EST_VC31','HC01_EST_VC32']]

In [5]:
merge1 = pd.merge(race2, ethnicity2, how='inner')

merge2 = pd.merge(merge1, economic2, how='inner')

merge3 = pd.merge(merge2, education2, how='inner')

merge4 = pd.merge(merge3, occupation2, how='inner')

merge4.head()

,GEO.id2,HD01_VD01,HD01_VD03,HD01_VD02,HD01_VD12,HC01_VC85,HC02_EST_VC18,HC01_EST_VC08,HC01_EST_VC31,HC01_EST_VC32
0,Id2,Estimate; Total:,Estimate; Population of one race: - White,Estimate; Not Hispanic or Latino:,Estimate; Hispanic or Latino:,Estimate; INCOME AND BENEFITS (IN 2016 INFLATI...,Percent; Estimate; Percent bachelor's degree o...,"Total; Estimate; Management, business, science...","Total; Estimate; Natural resources, constructi...","Total; Estimate; Natural resources, constructi..."
1,25001010100,2962,2517,2798,164,42228,49.7,0,54,0
2,25001010206,3168,2940,3158,10,48452,54.2,26,91,81
3,25001010208,1589,1563,1559,30,62051,53.8,0,102,16
4,25001010304,2107,2072,2084,23,61138,52.2,0,105,25


In [6]:
data1 = merge4.drop(merge4.index[0])

data1.index = range(len(data1))

data1.rename(columns={'HD01_VD01':'Total','HD01_VD03':'White','HD01_VD02':'Non_Hispanic',
                      'HD01_VD12':'Hispanic','HC01_VC85':'Income','HC02_EST_VC18':'Educated',
                      'HC01_EST_VC08':'Architecture and Engineering',
                      'HC01_EST_VC31':'Construction and Extraction',
                      'HC01_EST_VC32':'Installation, Maintenance, and Repair'}, 
             inplace = True)
data1.head()

,GEO.id2,Total,White,Non_Hispanic,Hispanic,Income,Educated,Architecture and Engineering,Construction and Extraction,"Installation, Maintenance, and Repair"
0,25001010100,2962,2517,2798,164,42228,49.7,0,54,0
1,25001010206,3168,2940,3158,10,48452,54.2,26,91,81
2,25001010208,1589,1563,1559,30,62051,53.8,0,102,16
3,25001010304,2107,2072,2084,23,61138,52.2,0,105,25
4,25001010306,2817,2636,2764,53,55455,46.7,0,90,10


In [7]:
other_race = [0 for n in range(data1.shape[0])]
i = 0

for (it, row) in data1.iterrows():
    other_race[i] = row['Total'] - row['White']
    i = i + 1

data1['Other_race'] = DataFrame(other_race)
data2 = data1.drop(['GEO.id2','Total'],axis=1)

data2.head()

,White,Non_Hispanic,Hispanic,Income,Educated,Architecture and Engineering,Construction and Extraction,"Installation, Maintenance, and Repair",Other_race
0,2517,2798,164,42228,49.7,0,54,0,445
1,2940,3158,10,48452,54.2,26,91,81,228
2,1563,1559,30,62051,53.8,0,102,16,26
3,2072,2084,23,61138,52.2,0,105,25,35
4,2636,2764,53,55455,46.7,0,90,10,181


In [8]:
#data2[(type(data2['Educated'])==str)]

data3 = data2[(data2.Educated != '-')]

data4 = data3[(data3.Income != '-')]

data4.index = range(len(data4))

In [9]:
data5 = data4.loc[:,['White','Other_race','Non_Hispanic','Hispanic','Income','Educated']]

In [10]:
for i in range(len(data5)):
    data5.loc[i,'White'] = int(data5.loc[i,'White'])
    data5.loc[i,'Other_race'] = int(data5.loc[i,'Other_race'])
    data5.loc[i,'Non_Hispanic'] = int(data5.loc[i,'Non_Hispanic'])
    data5.loc[i,'Hispanic'] = int(data5.loc[i,'Hispanic'])
    data5.loc[i,'Income'] = int(data5.loc[i,'Income'])
    data5.loc[i,'Educated'] = int(data5.loc[i,'Educated'])

In [11]:
data5.head()

,White,Other_race,Non_Hispanic,Hispanic,Income,Educated
0,2517,445,2798,164,42228,49
1,2940,228,3158,10,48452,54
2,1563,26,1559,30,62051,53
3,2072,35,2084,23,61138,52
4,2636,181,2764,53,55455,46


In [37]:
x_data = np.array(data5)

y_ac = data4['Architecture and Engineering'].tolist()
y_ce = data4['Construction and Extraction'].tolist()
y_imr = data4['Installation, Maintenance, and Repair'].tolist()

y_data_ac = np.array(y_ac)
y_data_ce = np.array(y_ce)
y_data_imr = np.array(y_imr)

In [285]:
#(x_train, x_test, y_train, y_test) = model_selection.train_test_split(x_data, y_data, test_size=0.2)

In [13]:
X_train = sm.add_constant(x_data)
model1 = sm.OLS(y_data_ac,X_train)
results1 = model1.fit()
print(results1.params)

[-3.68377604e+01  5.19146955e-03  3.41025321e-03  7.93866086e-03
  6.63061356e-04  1.93414538e-04  5.11314744e-01]


In [14]:
X_train = sm.add_constant(x_data)
model2 = sm.OLS(y_data_ce,X_train)
results2 = model2.fit()
print(results2.params)

[ 5.78327839e+01  1.65797241e-02 -9.31480271e-04  8.45438095e-03
  7.19386377e-03  3.51187263e-04 -1.97945258e+00]


In [15]:
X_train = sm.add_constant(x_data)
model3 = sm.OLS(y_data_imr,X_train)
results3 = model3.fit()
print(results3.params)

[ 2.55444173e+01  1.13997744e-02 -3.41979643e-03  5.70298346e-03
  2.27699485e-03  1.41718921e-04 -1.11635228e+00]


In [16]:
print(results1.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.369
Model:                            OLS   Adj. R-squared:                  0.367
Method:                 Least Squares   F-statistic:                     169.6
Date:                Fri, 07 Dec 2018   Prob (F-statistic):          3.82e-142
Time:                        19:29:28   Log-Likelihood:                -7436.0
No. Observations:                1455   AIC:                         1.488e+04
Df Residuals:                    1449   BIC:                         1.492e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -36.8378      3.949     -9.329      0.0

In [17]:
print(results2.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.427
Model:                            OLS   Adj. R-squared:                  0.425
Method:                 Least Squares   F-statistic:                     216.2
Date:                Fri, 07 Dec 2018   Prob (F-statistic):          1.83e-172
Time:                        19:29:45   Log-Likelihood:                -8021.2
No. Observations:                1455   AIC:                         1.605e+04
Df Residuals:                    1449   BIC:                         1.609e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         57.8328      5.904      9.796      0.0

In [18]:
print(results3.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.430
Model:                            OLS   Adj. R-squared:                  0.428
Method:                 Least Squares   F-statistic:                     218.4
Date:                Fri, 07 Dec 2018   Prob (F-statistic):          7.35e-174
Time:                        19:29:57   Log-Likelihood:                -7363.0
No. Observations:                1455   AIC:                         1.474e+04
Df Residuals:                    1449   BIC:                         1.477e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         25.5444      3.756      6.802      0.0

In [44]:
CIs1 = results1.conf_int()

notSignificant = (CIs1[:,0] < 0) & (CIs1[:,1] > 0)
for i in notSignificant:
    print(i)

False
False
False
False
True
False
False


In [46]:
CIs2 = results2.conf_int()

notSignificant = (CIs2[:,0] < 0) & (CIs2[:,1] > 0)
for i in notSignificant:
    print(i)

False
False
True
False
False
False
False


In [47]:
CIs3 = results3.conf_int()

notSignificant = (CIs3[:,0] < 0) & (CIs3[:,1] > 0)
for i in notSignificant:
    print(i)

False
False
False
False
True
False
False
